# Noisy Simulations

CUDA-Q provides the capabilites to perform noisy quantum circuit simulations. 

### Predefined Noise Channels

You can define the following types noise channels in CUDA-Q:

1. DepolarizationChannel - Quantum state Decays into a mix of basis states
2. AmplitudeDampingChannel - Qubit decays to ground state
3. BitFlipChannel - Bit flip occurs (180 degree around X axis)
4. PhaseFlipChannel - Phase flip occurs (180 degree rotation around Z axis)

The cell below demonstrates how to build a bit flip noise channel.  You can use the same code,and swap out any of the other noise channels.

Note that all noisy simulations must use the `density-matrix-cpu` backend.

In [2]:
import cudaq

cudaq.set_target('density-matrix-cpu')

noise = cudaq.NoiseModel() # Build an empty noise model

bit_flip = cudaq.BitFlipChannel(0.5) # Defines bit flip channel with a probability of error

noise.add_channel('x', [0], bit_flip) # Adds the bit flip error to X gates on qubit 0

@cudaq.kernel
def kernel():
    qubit = cudaq.qubit()
    x(qubit)
    mz(qubit)

noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 0:477 1:523 }

{ 1:1000 }



### Custom Noise Channels

Custom incoherent noise channels can be defined using linear, completely positive, and trace preserving maps. These maps are called Kraus operators $\{K_i\}$, which satisfy the condition $\sum_i K_i^{\dagger}K_i = \mathbb{I}$. The operators below define a bit flip error.


$$ K_0 = \sqrt{1-p} \Biggr(\begin{matrix} 
1 & 0 \\ 
0 & 1
\end{matrix}\Biggr)$$

$$ K_1 = \sqrt{p} \Biggr(\begin{matrix} 
0 & 1 \\ 
1 & 0
\end{matrix}\Biggr)$$


Using Kraus operators follows the same code pattern defined above, except for the operators are defined as 2x2 Numpy arrays and then provided as inputs to `cudaq.KrausChannel`

In [ ]:
import numpy as np

error_probability = .5

kraus_0 = np.sqrt(1 - error_probability) * np.array([[1.0, 0.0], [0.0, 1.0]],
                                                    dtype=np.complex128)

kraus_1 = np.sqrt(error_probability) * np.array([[0.0, 1.0], [1.0, 0.0]],
                                                dtype=np.complex128)

bitflip_channel = cudaq.KrausChannel([kraus_0, kraus_1])